# Prompt Declaration Language

Prompt engineering is difficult: minor variations in prompts have large impacts on the output of LLMs and prompts are model-dependent. In recent years <i> prompt programming languages </i> have emerged to bring discipline to prompt engineering. Many of them are embedded in an imperative language such as Python or TypeScript, making it difficult for users to directly interact with prompts and multi-turn LLM interactions.

The [Prompt Declaration Language (PDL)](https://ibm.github.io/prompt-declaration-language/) is a YAML-based declarative approach to prompt programming, where prompts are at the forefront. PDL facilitates model chaining and tool use, abstracting away the plumbing necessary for such compositions. It enables type checking of the input and output of models. PDL has been used with application patterns like RAG, CoT, ReAct, and an agent for solving SWE-bench. PDL is [open-source](https://github.com/IBM/prompt-declaration-language).

You can use PDL stand-alone or from a Python SDK. In a Jupyter notebook, a convenient extension lets you write PDL directly, without the need to write Python "boilerplate". It even provides color coding of the YAML declarations and in the cell output, model-generated text is rendered in green font, and tool-generated text is rendered in purple font. We will use this notebook extension here.

> **NOTE:** Internally, PDL uses [LiteLLM](https://www.litellm.ai/) to connect to a variety of model providers. This is an alternative to the [Langchain](https://www.langchain.com/) API we are using in other recipes. 

In [ ]:
import sys
assert sys.version_info >= (3, 11) and sys.version_info <= (3, 12), \
    f"Use Python 3.11 or 3.12 to run this notebook. You have {sys.version_info}"

In [ ]:
! pip install 'prompt-declaration-language[examples]<0.4.0'

## Model Call

In PDL, the user specifies step-by-step the shape of data they want to generate. In the following, the `text` construct indicates a text block containing a prompt and a model call. Implicitly, PDL builds a background conversational context (list of role/content) which is used to make model calls. Each model call uses the context built so far as its input prompt.


In [ ]:
from pdl.pdl import exec_str
model_path = "replicate/ibm-granite/granite-3.3-8b-instruct"
pdl = f"""
description: Model call
text:
- "What is the meaning of life?\n"
- model: {model_path}
  parameters:
    stop_sequences: "!"
    temperature: 0
"""

print(exec_str(pdl))

## Model Chaining
Model chaining, where the output of one model is used to query a second model (perhaps the same one), can be done by simply adding to the list of models to call declaratively.

In [ ]:
from pdl.pdl import exec_str
pdl = f"""
text:
- "\nSay it like a poem\n"
- model: {model_path}
- "\n\nWhat is the most important verse in this poem?\n"
- model: {model_path}
  parameters:
    temperature: 0
"""
scope = {"pdl_context": [{"role": "user", "content":"What is the meaning of life?"}, {"role": "assistant", "content": "The meaning of life is a philosophical and metaphysical question related to the purpose or significance of life or existence in general. This question has been asked for centuries and does not have a definitive answer. Some people find meaning through personal growth, relationships, love, or through contributing to the betterment of humanity. Others may find it through spirituality or religious beliefs. Ultimately, the meaning of life may be something personal and subjective."}]}
print(exec_str(prog=pdl, scope=scope))

## Chat Templates

The following example shows a full-fledged chatbot. In PDL, _roles_ are high level annotations and PDL takes care of applying the appropriate chat templates. This example illustrates the use of control structures such as the repeat-until block and reading from files or stdin with the read block. The chatbot repeatedly prompts the user for a query, which it submits to a model, and stops when the query is `quit`.

For your first query, type `What is APR?`. We'll discuss what happens to this string below.

> **NOTE:** Because user input is required for this example, we don't run it when running our CI tests. That's the purpose of the `if os.environ.get('GRANITE_TESTING') is None:` check.

In [ ]:
from pdl.pdl import exec_str
pdl = f"""
text:
- role: system
  text: You are Granite, an AI language model developed by IBM in 2024. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.
- "Type `quit` to exit this chatbot.\n"
- repeat:
    text:
    - read:
      message: ">>> "
      def: query
      contribute: [context]
    - model: {model_path}
  until: ${{ query == 'quit'}}
  join:
    with: "\n\n"
role: user
"""

import os
if os.environ.get('GRANITE_TESTING') is None:
    print(exec_str(pdl))
else:
    print("Since this is the test environment, we skip running the chat example.")

### A closer look

If your first query was `What is APR?`, then the first call to the model, the program submitted the following prompt:

```
<|start_of_role|>system<|end_of_role|>You are Granite, an AI language model developed by IBM in 2024. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.<|end_of_text|>
<|start_of_role|>user<|end_of_role|>Type `quit` to exit this chatbot.
What is APR?<|end_of_text|><|start_of_role|>assistant<|end_of_role|>
```

PDL takes care of applying the appropriate chat templates and tags, and builds the background context implicitly. Chat templates make your program easier to port across models, since you do not need to specify control tokens by hand. All you have to do is list the models they want to chain and PDL takes care of the rest.

## Data Pipeline

The following program shows a common prompting pattern: read some data, formulate a prompt using that data, submit to a model, and evaluate the results. 

Specifically, we formulate a prompt for code explanation. The program first defines two variables: `code`, which holds the data we read from `./data.yaml`, and `truth` for the ground truth read from `./ground_truth.txt`. It then prints out the source code, formulates a prompt with the data, and calls a model to get an explanation. Finally, a Python code block uses the [Levenshtein text distance metric](https://en.wikipedia.org/wiki/Levenshtein_distance) to evaluate the explanation against the ground truth. This pipeline can similarly be applied to an entire data set to produce a jsonl file.

In [ ]:
from pdl.pdl import exec_str

PDL = f"""
description: Code explanation example
defs:
  CODE:
    read: ./data.yaml
    parser: yaml
  TRUTH:
    read: ./ground_truth.txt
text:
- |
  Here is some info about the location of the function in the repo.
  repo:
  ${{ CODE.repo_info.repo }}
  path: ${{ CODE.repo_info.path }}
  Function_name: ${{ CODE.repo_info.function_name }}


  Explain the following code:
  ```
  ${{ CODE.source_code }}```
- model: {model_path}
  parameters:
      temperature: 0
  def: EXPLANATION
- |

  EVALUATION:
  The similarity (Levenshtein) between this answer and the ground truth is:
- def: EVAL
  lang: python
  code: |
    import textdistance
    expl = \"\"\"
    ${{ EXPLANATION }}
    \"\"\"
    truth = \"\"\"
    ${{ TRUTH }}
    \"\"\"
    result = textdistance.levenshtein.normalized_similarity(expl, truth)
"""


print(exec_str(PDL))


## Conclusion

Since prompts are at the forefront, PDL makes users more productive in their trial-and-error with LLMs. Try it!

For more information, visit 
https://github.com/IBM/prompt-declaration-language